In [1]:
import sys, random, os, json
sys.path.append(sys.path.append(os.path.join(os.getcwd(), '..')))
from datamart.augment import Augment
import pandas as pd

In [2]:
es_index = "datamart_all"

augment = Augment(es_index=es_index)

### Initialize a dataframe

In [3]:
old_df = pd.DataFrame(data={
    'city': ["los angeles", "New york", "Shanghai", "SAFDA", "manchester"],
    'country': ["US", "US", "China", "fwfb", "UK"],
})

print(old_df)

          city country
0  los angeles      US
1     New york      US
2     Shanghai   China
3        SAFDA    fwfb
4   manchester      UK


### Search metadata
Query by a column, which is query on variable.named_entities, by default, if a metadata matches more than half of cells in original dataframe, it is a hit. Can specify minimum should match with minimum_should_match parameter

In [4]:
hitted_metadatas = augment.query_by_column(
    col=old_df.loc[:, "city"], 
    minimum_should_match=len(old_df.loc[:, 'city'].unique().tolist())//2)

print(len(hitted_metadatas))

136


### Query by key value pairs

In [5]:
hitted_metadatas = augment.query_by_key_value_pairs(key_value_pairs=[
    ("description", "average")
])

print(len(hitted_metadatas))


9


### Query by temporal coverage

In [6]:
hitted_metadatas = augment.query_by_temporal_coverage(
    start="2018-09-23", 
    end="2018-09-30T00:00:00")

print(len(hitted_metadatas))

12588


With some ranking methods, say we want to augment with a specific metadata, datamart id 125610000

In [7]:
metadata = augment.query_by_datamart_id(datamart_id=125610000)[0]

In [8]:
# Take a look at some metadata
print(json.dumps(metadata, indent=2))

{
  "_index": "datamart_all",
  "_type": "_doc",
  "_id": "125610000",
  "_score": 1.0,
  "_source": {
    "datamart_id": 125610000,
    "title": "TAVG",
    "description": "Average temperature (tenths of degrees C)[Note that TAVG from source 'S' corresponds to an average for the period ending at 2400 UTC rather than local midnight]",
    "url": "https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt",
    "keywords": [
      "Average Temperature."
    ],
    "provenance": {
      "source": "noaa.org"
    },
    "materialization": {
      "python_path": "noaa_materializer",
      "arguments": {
        "type": "TAVG"
      }
    },
    "variables": [
      {
        "datamart_id": 125610001,
        "name": "date",
        "description": "the date of data",
        "semantic_type": [
          "https://metadata.datadrivendiscovery.org/types/Time"
        ],
        "temporal_coverage": {
          "start": "1874-10-13T00:00:00",
          "end": "2018-10-01T00:00:00"
        }
     

### Materialize dataset with constrains

In [9]:
# Get me subset of the dataset only related to my cities in old_df and time range from 2018-09-23 to 2018-09-30
new_df = augment.get_dataset(metadata=metadata["_source"], variables=None, constrains={
            "named_entity": old_df.loc[:, 'city'].unique().tolist(),
            "date_range": {
                "start": "2018-09-23T00:00:00",
                "end": "2018-09-30T00:00:00"
            }
        })

In [10]:
print(new_df.iloc[random.sample(range(1, new_df.shape[0]), 10), :])

                    date          stationid         city  TAVG
105  2018-09-23T00:00:00  GHCND:USW00014734     New york   168
128  2018-09-23T00:00:00  GHCND:CHM00058362     Shanghai   253
44   2018-09-26T00:00:00  GHCND:USR0000CCP9  los angeles   267
10   2018-09-23T00:00:00  GHCND:USR0000CWHH  los angeles   206
143  2018-09-23T00:00:00  GHCND:USW00014745   manchester   113
51   2018-09-26T00:00:00  GHCND:USW00023174  los angeles   190
147  2018-09-27T00:00:00  GHCND:USW00014745   manchester   187
1    2018-09-23T00:00:00  GHCND:USR0000CBEV  los angeles   206
141  2018-09-30T00:00:00  GHCND:CHM00058362     Shanghai   228
144  2018-09-24T00:00:00  GHCND:USW00014745   manchester   103


### Join
There are many ways of join between original dataframe and new dataframe.
Simplest solution is left join which will produce lots of rows for same city.
Default join of datamart implemented this method.

In [11]:
df = augment.join(
    left_df=old_df,
    right_df=new_df,
    left_columns=[0],  # city column index of old_df
    right_columns=[2],  # city column index of new_df
    joiner="default"
)
print(df)

            city country                 date          stationid   TAVG
0    los angeles      US  2018-09-23T00:00:00  GHCND:USR0000CACT  233.0
1    los angeles      US  2018-09-23T00:00:00  GHCND:USR0000CBEV  206.0
2    los angeles      US  2018-09-23T00:00:00  GHCND:USR0000CCHB  228.0
3    los angeles      US  2018-09-23T00:00:00  GHCND:USR0000CCHI  218.0
4    los angeles      US  2018-09-23T00:00:00  GHCND:USR0000CCLE  237.0
5    los angeles      US  2018-09-23T00:00:00  GHCND:USR0000CCP9  224.0
6    los angeles      US  2018-09-23T00:00:00  GHCND:USR0000CLTU  215.0
7    los angeles      US  2018-09-23T00:00:00  GHCND:USR0000CMAL  197.0
8    los angeles      US  2018-09-23T00:00:00  GHCND:USR0000CMIL  212.0
9    los angeles      US  2018-09-23T00:00:00  GHCND:USR0000CSFD  222.0
10   los angeles      US  2018-09-23T00:00:00  GHCND:USR0000CWHH  206.0
11   los angeles      US  2018-09-23T00:00:00  GHCND:USW00023129  214.0
12   los angeles      US  2018-09-23T00:00:00  GHCND:USW00023174

#### Aggregation
Join also can be performed based on aggregation.

In [16]:
# Aggregate on city
new_df_aggregated = new_df.groupby(["city"], as_index=False)["TAVG"].mean()
print(new_df_aggregated)

          city        TAVG
0     New york  188.416667
1     Shanghai  235.533333
2  los angeles  212.057692
3   manchester  134.000000


In [17]:
df = pd.merge(left=old_df, right=new_df_aggregated, left_on='city', right_on='city', how='left')
print(df)

          city country        TAVG
0  los angeles      US  212.057692
1     New york      US  188.416667
2     Shanghai   China  235.533333
3        SAFDA    fwfb         NaN
4   manchester      UK  134.000000


In [18]:
# Aggregate on city and date
new_df_aggregated = new_df.groupby(["city", "date"], as_index=False)["TAVG"].mean()
print(new_df_aggregated)

           city                 date        TAVG
0      New york  2018-09-23T00:00:00  174.000000
1      New york  2018-09-24T00:00:00  176.333333
2      New york  2018-09-25T00:00:00  184.666667
3      New york  2018-09-26T00:00:00  239.333333
4      New york  2018-09-27T00:00:00  206.666667
5      New york  2018-09-28T00:00:00  170.000000
6      New york  2018-09-29T00:00:00  182.000000
7      New york  2018-09-30T00:00:00  174.333333
8      Shanghai  2018-09-23T00:00:00  254.500000
9      Shanghai  2018-09-24T00:00:00  210.000000
10     Shanghai  2018-09-25T00:00:00  233.000000
11     Shanghai  2018-09-26T00:00:00  234.500000
12     Shanghai  2018-09-27T00:00:00  236.000000
13     Shanghai  2018-09-28T00:00:00  239.000000
14     Shanghai  2018-09-29T00:00:00  232.500000
15     Shanghai  2018-09-30T00:00:00  232.000000
16  los angeles  2018-09-23T00:00:00  216.384615
17  los angeles  2018-09-24T00:00:00  195.923077
18  los angeles  2018-09-25T00:00:00  202.923077
19  los angeles  201

In [19]:
df = pd.merge(left=old_df, right=new_df_aggregated, left_on='city', right_on='city', how='left')
print(df)

           city country                 date        TAVG
0   los angeles      US  2018-09-23T00:00:00  216.384615
1   los angeles      US  2018-09-24T00:00:00  195.923077
2   los angeles      US  2018-09-25T00:00:00  202.923077
3   los angeles      US  2018-09-26T00:00:00  227.538462
4   los angeles      US  2018-09-27T00:00:00  238.692308
5   los angeles      US  2018-09-28T00:00:00  212.615385
6   los angeles      US  2018-09-29T00:00:00  192.692308
7   los angeles      US  2018-09-30T00:00:00  209.692308
8      New york      US  2018-09-23T00:00:00  174.000000
9      New york      US  2018-09-24T00:00:00  176.333333
10     New york      US  2018-09-25T00:00:00  184.666667
11     New york      US  2018-09-26T00:00:00  239.333333
12     New york      US  2018-09-27T00:00:00  206.666667
13     New york      US  2018-09-28T00:00:00  170.000000
14     New york      US  2018-09-29T00:00:00  182.000000
15     New york      US  2018-09-30T00:00:00  174.333333
16     Shanghai   China  2018-0

We can also unstack new datagrame to form more columns and so that we will not produce extra rows

In [20]:
new_df_unstacked = new_df.groupby(["city", "date"])["TAVG"].mean().unstack().reset_index(level=['city'])
df = pd.merge(left=old_df, right=new_df_unstacked, left_on='city', right_on='city', how='outer')
print(df)

          city country  2018-09-23T00:00:00  2018-09-24T00:00:00  \
0  los angeles      US           216.384615           195.923077   
1     New york      US           174.000000           176.333333   
2     Shanghai   China           254.500000           210.000000   
3        SAFDA    fwfb                  NaN                  NaN   
4   manchester      UK           113.000000           103.000000   

   2018-09-25T00:00:00  2018-09-26T00:00:00  2018-09-27T00:00:00  \
0           202.923077           227.538462           238.692308   
1           184.666667           239.333333           206.666667   
2           233.000000           234.500000           236.000000   
3                  NaN                  NaN                  NaN   
4            92.000000           198.000000           187.000000   

   2018-09-28T00:00:00  2018-09-29T00:00:00  2018-09-30T00:00:00  
0           212.615385           192.692308           209.692308  
1           170.000000           182.000000     